In [1]:
#default_exp steam_data

In [2]:
#export 

import pandas as pd
from sklearn import compose, feature_extraction, metrics
from functools import reduce, partial

In [3]:
#export

STEAM_GAME_METADATA_PATH = 'data/steam_games_metadata/steam_games.csv'

In [4]:
%cd ..

/home/kuba/Projects/game_recommender


In [5]:
!ls data/steam_games_metadata/

steam_games.csv


In [6]:
raw_steam_df = pd.read_csv(STEAM_GAME_METADATA_PATH)

# Basic information

In [7]:
raw_steam_df.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


In [8]:
raw_steam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40833 entries, 0 to 40832
Data columns (total 20 columns):
url                         40833 non-null object
types                       40831 non-null object
name                        40817 non-null object
desc_snippet                27612 non-null object
recent_reviews              2706 non-null object
all_reviews                 28470 non-null object
release_date                37654 non-null object
developer                   40490 non-null object
publisher                   35733 non-null object
popular_tags                37888 non-null object
game_details                40313 non-null object
languages                   40797 non-null object
achievements                12194 non-null float64
genre                       40395 non-null object
game_description            37920 non-null object
mature_content              2897 non-null object
minimum_requirements        21069 non-null object
recommended_requirements    21075 non-null

# Content without names

These correspond to steam product bundles. These aren't really games so we drop them. 

In [9]:
raw_steam_df[raw_steam_df['name'].isna()].head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
704,https://store.steampowered.com/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$29.99,$23.99
4847,https://store.steampowered.com/bundle/6191/The...,app,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,About this bundle Get all extra characters i...,NaN,NaN,NaN,NaN,$18.84
6381,https://store.steampowered.com/bundle/7125/Fac...,app,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,About this bundle This bundle contains thre...,NaN,NaN,NaN,NaN,$2.67
7869,https://store.steampowered.com/bundle/10241/Da...,app,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$53.96
9615,https://store.steampowered.com/bundle/8069/Hap...,app,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,About this bundle Happy Bat VR Bundle contai...,NaN,NaN,NaN,NaN,$11.03


# Data cleaning

We remove entries that have a null in any of interesting fields

In [10]:
#export

def clean_steam_df(raw_steam_df):
    text_columns = [
        'name',
        'game_description',
        'game_details',
        'popular_tags',
        'desc_snippet',
        'genre'
    ]
    invalid_indices = reduce(
        lambda s1, s2: s1 | s2,
        [raw_steam_df[c].isna() for c in text_columns])


    tag_cols = ['popular_tags', 'game_details', 'genre']

    steam_df = raw_steam_df.copy()[~invalid_indices]
    steam_df.index = pd.RangeIndex(0, len(steam_df))
    for c in tag_cols:
        steam_df[c] = steam_df[c].str.lower()
    return steam_df


def load_steam_df(steam_games_metadata_path=STEAM_GAME_METADATA_PATH):
    return clean_steam_df(pd.read_csv(steam_games_metadata_path))


def get_games_by_name(steam_df, substr):
    game_names = steam_df['name'][steam_df['name'].str.contains(substr)]
    return steam_df[steam_df['name'].isin(game_names)]

In [11]:
steam_df = load_steam_df() 

In [12]:
chosen_games_substring = 'S.T.A.L.K'

In [13]:
chosen_games_df = get_games_by_name(steam_df, chosen_games_substring) 

In [14]:
chosen_games_df['name']

376     S.T.A.L.K.E.R.: Shadow of Chernobyl
1276        S.T.A.L.K.E.R.: Call of Pripyat
7253              S.T.A.L.K.E.R.: Clear Sky
Name: name, dtype: object

In [15]:
chosen_games_df.iloc[0]['popular_tags']

'atmospheric,post-apocalyptic,open world,fps,survival,masterpiece,action,rpg,survival horror,shooter,horror,first-person,singleplayer,classic,based on a novel,moddable,adventure,sci-fi,inventory management,difficult'

In [16]:
chosen_games_df.iloc[0]['game_details']

'single-player,multi-player'

In [17]:
chosen_games_df.iloc[0]['genre']

'action,rpg'

In [18]:
chosen_games_df['game_description'].iloc[0]

" About This Game In 1986, the world's worst nuclear disaster occurred at the Chernobyl Nuclear Power Plant. Soviet authorities established an 'Exclusion Zone' around, but a second explosion hit the reactor in 2006, creating The Zone as we know it – dangerous place, filled with mutated creatures, deadly radiation, and a strange, anomalous energy. The Zone was cordoned off by the military, who would shoot on sight anyone foolish enough to get inside.  Year 2012. The Zone is still a dangerous place and a threat to all mankind. Mercenaries, bounty hunters and explorers ventured further and further into the heart of the Zone driven by reports of strange 'artifacts' imbued with anomalous energy. To sell them on the black market or trying to find the 'truth' behind the Zone. Whatever their motivation, over time these individuals - Scavengers, Trespassers, Adventurers, Loners, Killers, Explorers and Robbers - have become known as S.T.A.L.K.E.R.s.  You will have to find your own way to survive

In [19]:
chosen_games_df['popular_tags'].iloc[0]

'atmospheric,post-apocalyptic,open world,fps,survival,masterpiece,action,rpg,survival horror,shooter,horror,first-person,singleplayer,classic,based on a novel,moddable,adventure,sci-fi,inventory management,difficult'

In [20]:
game_descriptions = steam_df['game_description']

# Finding games with similar descriptions

In [21]:
from collections import defaultdict

In [22]:
#export

def split_by_comma_tokenizer(text):
    return text.split(',') 


def _regroup_weights(weights, column_groups):
    assert len(weights) == len(column_groups), 'specify weight for each column group'
    return {
        col: weight / len(col_group)
        for (weight, col_group) in zip(weights, column_groups)
        for col in col_group
    }


def make_df_column_vectorizer(
        column_groups,
        vectorizer_classes=feature_extraction.text.TfidfVectorizer,
        weights=None,
        tokenizers=None
    ):
    if type(vectorizer_classes) is not list:
        vectorizer_classes = [vectorizer_classes for __ in range(len(column_groups))]
    if tokenizers is None:
        tokenizers = [None for __ in range(len(column_groups))]
    vectorizers = [vectorizer_class(tokenizer=tokenizer) for (vectorizer_class, grp, tokenizer) in zip(vectorizer_classes, column_groups, tokenizers)] 
    if type(weights) is list:
        weights = _regroup_weights(weights, column_groups)
    
    return compose.ColumnTransformer(
        [
            (col + '_tfidf', vectorizer, col)
            for columns, vectorizer in zip(column_groups, vectorizers)
            for col in columns
        ],
        transformer_weights=weights
    )

In [23]:
vectorizer = make_df_column_vectorizer([
        ['popular_tags'],
        ['game_description', 'desc_snippet'],
    ],
    vectorizer_classes=partial(
            feature_extraction.text.CountVectorizer,
            binary=True,
            min_df=5
    ),
    tokenizers=[split_by_comma_tokenizer, None]
)

steam_metadata_vectors = vectorizer.fit_transform(steam_df)

In [24]:
steam_df.shape

(24210, 20)

In [25]:
steam_metadata_vectors.shape

(24210, 31455)

In [26]:
#export

def similar_games(
        chosen_games_df,
        metadata_df,
        metadata_vectors,
        n_similar=20,
        distance=metrics.pairwise.cosine_distances):
    def get_names_from_indices(indices):
        return metadata_df['name'].iloc[indices].values
    
    chosen_metadata_vectors = metadata_vectors[chosen_games_df.index]
    mean_metadata_vector = chosen_metadata_vectors.mean(axis=0)
    similarity_sorted_indices = distance(chosen_metadata_vectors, metadata_vectors).argsort(axis=1)
    mean_metadata_similarity_sorted_indices = distance(mean_metadata_vector.reshape(1,-1), metadata_vectors).argsort(axis=1)
    
    mean_similar = get_names_from_indices(mean_metadata_similarity_sorted_indices[0][:n_similar])
    cols = {name: get_names_from_indices(similarity_sorted_indices[i][1:n_similar+1]) for (i, name) in enumerate(chosen_games_df['name'].values)}
    cols['mean'] = mean_similar
    df = pd.DataFrame(cols)
    return df

In [27]:
similar_games(chosen_games_df, steam_df, steam_metadata_vectors)

,S.T.A.L.K.E.R.: Shadow of Chernobyl,S.T.A.L.K.E.R.: Call of Pripyat,S.T.A.L.K.E.R.: Clear Sky,mean
0,S.T.A.L.K.E.R.: Call of Pripyat,S.T.A.L.K.E.R.: Clear Sky,S.T.A.L.K.E.R.: Call of Pripyat,S.T.A.L.K.E.R.: Shadow of Chernobyl
1,Fear The Wolves,S.T.A.L.K.E.R.: Shadow of Chernobyl,CHERNOBYL HISTORY OF NUCLEAR DISASTER,S.T.A.L.K.E.R.: Call of Pripyat
2,S.T.A.L.K.E.R.: Clear Sky,Zone Anomaly,S.T.A.L.K.E.R.: Shadow of Chernobyl,S.T.A.L.K.E.R.: Clear Sky
3,Phoning Home,STAR WARS™ Republic Commando™,Zone Anomaly,Zone Anomaly
4,Zone Anomaly,Dungeon Siege,Masked Forces: Zombie Survival,CHERNOBYL HISTORY OF NUCLEAR DISASTER
5,Exanima,Halo: The Master Chief Collection,Doom 3: BFG Edition,Doom 3: BFG Edition
6,Metro 2033 Redux,Call of Duty: World at War,Convoy,Shadows of Kurgansk
7,Red Faction,"STAR WARS™ Battlefront (Classic, 2004)",Quiet Sleep,Quake IV
8,九劫曲：诅咒之地 NINE TRIALS,State of Decay,Tom Clancy’s The Division™,Survivalist
9,Don't Starve Together,Call of Duty®: Modern Warfare® 3,Zombies In The Dark,Dispatcher


In [28]:
from sklearn import decomposition

n_components = 250


tsvd = decomposition.TruncatedSVD(n_components=500)
nmf = decomposition.NMF(n_components=100)

In [29]:
%%time
steam_metadata_reduced_vectors = tsvd.fit_transform(steam_metadata_vectors)

CPU times: user 3min 33s, sys: 1min 9s, total: 4min 42s
Wall time: 57.3 s


# Searching by SVD reduced data

In [30]:
similar_games(chosen_games_df, steam_df, steam_metadata_reduced_vectors)

,S.T.A.L.K.E.R.: Shadow of Chernobyl,S.T.A.L.K.E.R.: Call of Pripyat,S.T.A.L.K.E.R.: Clear Sky,mean
0,Rain World,Halo: The Master Chief Collection,Quiet Sleep,S.T.A.L.K.E.R.: Shadow of Chernobyl
1,Red Faction,STAR WARS™ Republic Commando™,P.A.S.,S.T.A.L.K.E.R.: Call of Pripyat
2,Phoning Home,Call of Duty: World at War,F.E.A.R.,S.T.A.L.K.E.R.: Clear Sky
3,Fear The Wolves,Half-Life 2: Episode Two,CHERNOBYL HISTORY OF NUCLEAR DISASTER,E.Y.E: Divine Cybermancy
4,Tom Clancy's Splinter Cell®,Call of Duty®: Modern Warfare® 2,Red Faction®: Armageddon™,F.E.A.R. 2: Project Origin
5,Metro 2033 Redux,F.E.A.R. 3,Call of Cthulhu: Shadow of the Comet,Metro 2033 Redux
6,F.E.A.R. 2: Project Origin,Dead Realm,Deus Ex: Game of the Year Edition,A.I.M.2 Clan Wars
7,Nosferatu: The Wrath of Malachi,Beasts of Bermuda,Outlast 2,Return to Castle Wolfenstein
8,Exanima,Tom Clancy's Splinter Cell®,Syndrome,Tom Clancy's Splinter Cell®
9,E.Y.E: Divine Cybermancy,The Fifth Day,Zone Anomaly,Affliction


# Searching by NMF reduced data

In [ ]:
%%time
steam_metadata_nmf_vectors = nmf.fit_transform(steam_metadata_vectors)

In [ ]:
similar_games(chosen_games_df, steam_df, steam_metadata_nmf_educed_vectors)